In [1]:
# ler todos os arquivos csv do diretório e guardar em um objeto
# import required modules
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd
from collections import Counter

diretorio = r'Bases/Arquivos_Limpos/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Layout CNPJ
CNPJ = {"EMPRE" : ['CNPJ_BASE', 'RAZAO_SOCIAL', 'NATUREZA_JURIDICA' , 'QUALIFICACAO' ,
          'VALOR_CAPITAL_SOCIAL' , 'PORTE_EMPRESA'  , 'ENTE_FEDERATIVO'],

"ESTABELE" : ['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV' , 'MATRIZ_FILIAL'  , 'NOME_FANTASIA' , 'SITUACAO_CADASTRAL' ,
              'DATA_SITUACAO_CADASTRAL'  , 'MOTIVO_SITUACAO_CADASTRAL'  , 'CIDADE_EXTERIOR'  , 'PAIS'  , 'DATA_INICIO_ATIVIDADE'  ,
              'CNAE_PRINCIPAL'  , 'CNAE_SECUNDARIO' , 'TIPO_LOGRADOURO'  , 'LOGRADOURO'  , 'NUMERO'  , 'COMPLEMENTO' ,
              'BAIRRO'  , 'CEP'  , 'UF'  , 'MUNICIPIO'  , 'DDD1'  , 'TELEFONE1'  , 'DDD2'  , 'TELEFONE2'  ,
              'DDD_FAX'  , 'FAX'  , 'EMAIL'  , 'SITUACAO_ESPECIAL'  , 'DATA_SITUACAO_ESPECIAL'],  

"SIMPLES" : ['CNPJ_BASE'  , 'OPCAO_SIMPLES'  , 'DATA_OPCAO_SIMPLES'  , 'DATA_EXCLUSAO_SIMPLES'  ,
             'OPCAO_MEI'  , 'DATA_OPCAO_MEI'  , 'DATA_EXCLUSAO_MEI'],

"SOCIO" : ['CNPJ_BASE'  , 'TIPO'  , 'NOME'  , 'CPF_CNPJ'  , 'QUALIFICACAO'  , 'DATA_ENTRADA' ,
            'PAIS' , 'REPRESENTANTE'  , 'NOME_REPRESENTANTE'  , 'QUALIFICACAO_REPRESENTANTE'  , 'FAIXA_ETARIA'  ],

"PAIS" : ['CODIGO_PAIS' , 'PAIS'],

"MUNIC" : ['MUNICIPIO'  , 'MUNICIPIO'  ],

"QUALS" : ['QUALIFICACAO', 'QUALIFICACAO' ],

"NATJU" : ['NATUREZA_JURIDICA'  , 'NATUREZA_JURIDICA'],

"MOTI" : ['MOTIVO_SITUACAO_CADASTRAL'  , 'MOTIVO_SITUACAO_CADASTRAL'  ],

"CNAE" : ['CODIGO_CNAE'  , 'CNAE' ]}


In [2]:
CNAES = {5611201:'Restaurantes e similares',
        5611203:'Lanchonetes casas de chá de sucos e similares',
        5611204:'Bares e outros estabelecimentos especializados em servir bebidas sem entretenimento',
        5611205:'Bares e outros estabelecimentos especializados em servir bebidas com entretenimento',
        5612100: 'Serviços ambulantes de alimentação'}

lista_cnae = []
for cnae in CNAES.keys():
    lista_cnae.append(cnae)

In [3]:
cartoes = pd.read_excel(r"C:\Users\ABRASEL NACIONAL\Documents\CNPJ_PROGRAMATICA\Painel_Leads\BaseCsv\ATUALIZADO_December_Estabelecimentos_BenVisaVale.xlsx")
cartoes.head(5)
lista_lojas_cartoes = cartoes['Estabelecimento'].map(str).to_list()
lista_enderecos_cartoes = cartoes['Endereço'].map(str).to_list()
del cartoes

In [5]:

df_1 = pd.read_csv(f'{diretorio}{all_files[0]}', names=['NOME_FANTASIA','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO'], sep=';',encoding='utf-8', usecols=[4, 14, 15, 16, 17], low_memory=False)
df_1.head(5)


,NOME_FANTASIA,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO
0,AGO RESTOBAR,CLOUTIDES PEREIRA ROCHA,SN,NaN,BAIRRO CEARENSE
1,Indisponível,GETULIO VARGAS,158,NaN,CENTRO
2,PROMESSA DE DEUS,SALAO,681,BLOCO CASA,RUA AUGUSTINHO ALEGRE CHIC
3,PESQUE & PAGUE BOM PESQUEIRO,RODOVIA PR364,SN,NaN,CABECEIRA DO ARROIO GRANDE
4,BAR DO BEM,PARA,440,NaN,PACOVAL


In [9]:
NOMEBEN_OK = []
for row in df_1['NOME_FANTASIA'].map(str):
    if row in lista_lojas_cartoes:
        NOMEBEN_OK.append(1)
    else:
        NOMEBEN_OK.append(0)
print(Counter(NOMEBEN_OK).keys())
print(Counter(NOMEBEN_OK).values())

In [8]:
NOMEBEN_OK = []
for drow in df_1['NOME_FANTASIA'].map(str):
    for crow in lista_lojas_cartoes:
        if drow in crow:
            NOMEBEN_OK.append(1)
        else:
            NOMEBEN_OK.append(0)
print(Counter(NOMEBEN_OK).keys())
print(Counter(NOMEBEN_OK).values())

In [6]:
TESTE = ['RUA CARACA 934, 934 CASA NOSSA SENHORA DA CONCEIÇÃO']	

if '934 CASA' in str(TESTE):
    print('SIM')
else:
    print('NÃO')

SIM


In [18]:
print(Counter(NOMEBEN_OK).keys())
print(Counter(NOMEBEN_OK).values())

dict_keys([0, 1])
dict_values([69957, 4948])


In [10]:
LOGRADOUROBEN_OK = []
for row in df_1['LOGRADOURO'].map(str):
    if row in lista_enderecos_cartoes:
        LOGRADOUROBEN_OK.append(1)
    else:
        LOGRADOUROBEN_OK.append(0)
print(Counter(LOGRADOUROBEN_OK).keys())
print(Counter(LOGRADOUROBEN_OK).values())

In [17]:
print(Counter(LOGRADOUROBEN_OK).keys())
print(Counter(LOGRADOUROBEN_OK).values())

dict_keys([0])
dict_values([74905])


In [11]:
NUMEROBEN_OK = []
for row in df_1['NUMERO'].map(str):
    if row in lista_enderecos_cartoes:
        NUMEROBEN_OK.append(1)
    else:
        NUMEROBEN_OK.append(0)
print(Counter(NUMEROBEN_OK).keys())
print(Counter(NUMEROBEN_OK).values())

In [16]:
print(Counter(NUMEROBEN_OK).keys())
print(Counter(NUMEROBEN_OK).values())

dict_keys([0])
dict_values([74905])


In [ ]:
COMPLEMENTOBEN_OK = []
for row in df_1['COMPLEMENTO'].map(str):
    if row in lista_enderecos_cartoes:
        COMPLEMENTOBEN_OK.append(1)
    else:
        COMPLEMENTOBEN_OK.append(0)
print(Counter(COMPLEMENTOBEN_OK).keys())
print(Counter(COMPLEMENTOBEN_OK).values())

In [15]:
print(Counter(COMPLEMENTOBEN_OK).keys())
print(Counter(COMPLEMENTOBEN_OK).values())

dict_keys([0])
dict_values([74905])


In [14]:
BAIRROBEM_OK = []
for row in df_1['BAIRRO'].map(str):
    if row in lista_enderecos_cartoes:
        BAIRROBEM_OK.append(1)
    else:
        BAIRROBEM_OK.append(0)
print(Counter(BAIRROBEM_OK).keys())
print(Counter(BAIRROBEM_OK).values())

dict_keys([0])
dict_values([74905])
